In [1]:
a = [1,2]
b = ['a','b']

In [2]:
dict(zip(a,b))

{1: 'a', 2: 'b'}

In [7]:
import numpy as np
def read_align(path_to_align = None):
    with open(path_to_align, 'r') as f:
        lines = f.readlines()	
    align = [(int(y[0])/1000, int(y[1])/1000, y[2]) for y in [x.strip().split(" ") for x in lines]]
    words = []
    words.append('sil')
    for i in range(75):
    	for j in align:
	    	if i > j[0] and i <= j[1]:
	    		words.append(j[2])
    dict_align  = np.array(words)
    return dict_align

In [18]:
import dlib
from skvideo.io import vread
from scipy.misc import imresize
def read_video(path, face_predictor_path, verbose=False):
    if verbose:
        print("loading: " + path)
    video = vread(path)
    frames = np.array([frame for frame in video])
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(face_predictor_path)
    mouth_video = get_frames_mouth(detector, predictor, frames)

    return np.array(mouth_video)

def get_frames_mouth(detector, predictor, frames):
        MOUTH_WIDTH = 100
        MOUTH_HEIGHT = 50
        HORIZONTAL_PAD = 0.19
        normalize_ratio = None
        mouth_frames = []
        for frame in frames:
            dets = detector(frame, 1)
            shape = None
            for k, d in enumerate(dets):
                shape = predictor(frame, d)
                i = -1
            if shape is None: # Detector doesn't detect face, just return as is
                return frames
            mouth_points = []
            for part in shape.parts():
                i += 1
                if i < 48: # Only take mouth region
                    continue
                mouth_points.append((part.x,part.y))
            np_mouth_points = np.array(mouth_points)

            mouth_centroid = np.mean(np_mouth_points[:, -2:], axis=0)

            if normalize_ratio is None:
                mouth_left = np.min(np_mouth_points[:, :-1]) * (1.0 - HORIZONTAL_PAD)
                mouth_right = np.max(np_mouth_points[:, :-1]) * (1.0 + HORIZONTAL_PAD)

                normalize_ratio = MOUTH_WIDTH / float(mouth_right - mouth_left)

            new_img_shape = (int(frame.shape[0] * normalize_ratio), int(frame.shape[1] * normalize_ratio))
            resized_img = imresize(frame, new_img_shape)

            mouth_centroid_norm = mouth_centroid * normalize_ratio

            mouth_l = int(mouth_centroid_norm[0] - MOUTH_WIDTH / 2)
            mouth_r = int(mouth_centroid_norm[0] + MOUTH_WIDTH / 2)
            mouth_t = int(mouth_centroid_norm[1] - MOUTH_HEIGHT / 2)
            mouth_b = int(mouth_centroid_norm[1] + MOUTH_HEIGHT / 2)

            mouth_crop_image = resized_img[mouth_t:mouth_b, mouth_l:mouth_r]

            mouth_frames.append(mouth_crop_image)
        return mouth_frames

In [12]:
import os
#from align import read_align
#from video import read_video

CURRENT_PATH = '/home/ubuntu/Project/machine-lip-reading/preprocessing'
DATA_PATH = CURRENT_PATH + '/../data'
PREDICTOR_PATH = CURRENT_PATH + '/shape_predictor_68_face_landmarks.dat'


def load_data(verbose=False, framebyframe=False):
    for root, dirs, files in os.walk(DATA_PATH):
        for name in files:
            if '.mpg' in name:
                if verbose is True:
                    print("reading: " + root)

                video = read_video(os.path.join(root, name), PREDICTOR_PATH)
                words = read_align(os.path.join(root, '../align/', name.split(".")[0] + ".align"))
               
                if verbose is True:
                    print("video shape:", video.shape)
                    print("alignments shape:", words.shape)

                if framebyframe:
                    for frame, word in zip(video, words):
                        yield frame, word
                else:
                    yield video, words





In [19]:
for img, word in load_data(verbose=True, framebyframe=True):
    print(img, word)



reading: /home/ubuntu/Project/machine-lip-reading/preprocessing/../data/s7/video


/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


('video shape:', (75, 50, 100, 3))
('alignments shape:', (75,))
(array([[[161, 122,  92],
        [157, 121,  87],
        [151, 116,  80],
        ...,
        [133,  98,  69],
        [103,  74,  43],
        [ 81,  58,  28]],

       [[172, 132, 103],
        [171, 132,  98],
        [161, 124,  89],
        ...,
        [130,  95,  62],
        [ 99,  70,  37],
        [ 85,  61,  32]],

       [[179, 138, 109],
        [179, 141, 107],
        [169, 131,  96],
        ...,
        [123,  88,  54],
        [ 96,  67,  34],
        [ 90,  67,  37]],

       ...,

       [[ 83,  69,  51],
        [ 83,  71,  53],
        [ 78,  67,  48],
        ...,
        [127, 116,  81],
        [121, 110,  75],
        [121, 110,  76]],

       [[ 84,  70,  52],
        [ 85,  72,  54],
        [ 77,  64,  46],
        ...,
        [136, 124,  90],
        [129, 117,  83],
        [127, 115,  82]],

       [[ 85,  71,  53],
        [ 86,  72,  54],
        [ 75,  61,  43],
        ...,
        [

('video shape:', (75, 50, 100, 3))
('alignments shape:', (75,))
(array([[[157, 111,  72],
        [170, 123,  84],
        [185, 134,  95],
        ...,
        [ 92,  70,  32],
        [ 86,  64,  23],
        [ 93,  72,  29]],

       [[156, 110,  71],
        [169, 122,  83],
        [184, 133,  94],
        ...,
        [ 93,  71,  33],
        [ 86,  65,  24],
        [ 94,  73,  30]],

       [[154, 108,  69],
        [166, 119,  80],
        [182, 131,  92],
        ...,
        [ 91,  72,  35],
        [ 86,  67,  27],
        [ 95,  77,  35]],

       ...,

       [[ 99,  75,  42],
        [ 98,  74,  43],
        [ 93,  72,  45],
        ...,
        [ 87,  70,  33],
        [ 93,  76,  36],
        [100,  82,  40]],

       [[ 99,  76,  43],
        [ 98,  76,  43],
        [ 95,  75,  45],
        ...,
        [ 85,  69,  31],
        [ 91,  74,  34],
        [ 97,  81,  39]],

       [[ 99,  77,  43],
        [ 98,  77,  44],
        [ 97,  77,  45],
        ...,
        [

('video shape:', (75, 50, 100, 3))
('alignments shape:', (75,))
(array([[[168, 116,  97],
        [172, 115,  95],
        [176, 114,  92],
        ...,
        [ 89,  63,  35],
        [ 65,  40,  18],
        [ 56,  35,  11]],

       [[161, 110,  91],
        [172, 116,  96],
        [174, 113,  91],
        ...,
        [ 64,  41,  14],
        [ 57,  34,  12],
        [ 51,  31,   8]],

       [[138,  92,  74],
        [164, 113,  95],
        [173, 116,  97],
        ...,
        [ 47,  28,   5],
        [ 48,  30,   9],
        [ 46,  30,   7]],

       ...,

       [[110,  83,  48],
        [114,  88,  52],
        [120,  96,  61],
        ...,
        [ 81,  65,  33],
        [ 80,  64,  32],
        [ 77,  61,  29]],

       [[118,  91,  55],
        [113,  87,  51],
        [112,  88,  53],
        ...,
        [ 84,  68,  35],
        [ 81,  65,  32],
        [ 77,  61,  28]],

       [[136, 110,  71],
        [125, 100,  62],
        [112,  88,  51],
        ...,
        [

('video shape:', (75, 50, 100, 3))
('alignments shape:', (75,))
(array([[[172, 115,  96],
        [175, 116,  94],
        [179, 119,  96],
        ...,
        [ 54,  29,   4],
        [ 51,  29,   9],
        [ 48,  29,  11]],

       [[173, 117,  98],
        [177, 118,  97],
        [182, 121, 100],
        ...,
        [ 43,  19,   1],
        [ 46,  26,   7],
        [ 47,  31,  12]],

       [[165, 112,  96],
        [177, 119, 103],
        [185, 124, 108],
        ...,
        [ 41,  26,   6],
        [ 43,  32,  15],
        [ 44,  35,  18]],

       ...,

       [[111,  84,  52],
        [116,  90,  59],
        [115,  90,  60],
        ...,
        [ 79,  69,  32],
        [ 73,  62,  27],
        [ 76,  64,  29]],

       [[108,  81,  46],
        [107,  82,  46],
        [107,  83,  48],
        ...,
        [ 81,  71,  34],
        [ 79,  68,  33],
        [ 77,  65,  31]],

       [[114,  88,  51],
        [108,  83,  47],
        [104,  80,  45],
        ...,
        [

('video shape:', (75, 50, 100, 3))
('alignments shape:', (75,))
(array([[[163, 112,  73],
        [173, 123,  81],
        [180, 130,  87],
        ...,
        [ 88,  74,  28],
        [ 83,  72,  26],
        [ 78,  61,  18]],

       [[164, 112,  72],
        [174, 123,  81],
        [182, 131,  88],
        ...,
        [ 89,  75,  30],
        [ 80,  69,  25],
        [ 75,  59,  17]],

       [[165, 112,  71],
        [176, 123,  80],
        [185, 131,  88],
        ...,
        [ 88,  75,  32],
        [ 78,  66,  25],
        [ 73,  56,  16]],

       ...,

       [[ 96,  72,  39],
        [ 98,  74,  41],
        [ 97,  73,  40],
        ...,
        [ 91,  81,  32],
        [ 97,  87,  36],
        [ 97,  87,  36]],

       [[ 96,  72,  39],
        [ 98,  74,  41],
        [ 97,  73,  41],
        ...,
        [ 89,  79,  31],
        [ 95,  85,  34],
        [ 98,  88,  37]],

       [[ 95,  71,  38],
        [ 96,  72,  39],
        [ 97,  73,  41],
        ...,
        [

KeyboardInterrupt: 